In [1]:
!pip install pandas numpy scikit-learn nltk sentence-transformers


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
!pip install PyPDF2 langchain

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 5.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/2.0 MB 3.9 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 3.9 MB/s eta 0:00:01
   ------------------------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import PyPDF2
import re
import os
import pandas as pd # pandas is useful for organizing extracted data later

# --- Configuration ---
# Set the directory where your PDF books are located
# Make sure your PDF files are in a 'books' subfolder within your ChatBotProject folder.
pdf_dir = 'books'

# List of PDF file names you want to process
pdf_files = [
    'C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf',
    'C:/Users/Ayush Jindal/Downloads/Wallach.pdf',
    'C:/Users/Ayush Jindal/Downloads/Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf',
    "C:/Users/Ayush Jindal/Downloads/guidelines-on-mental-health-promotive-and-preventive-interventions-for-adolescents-hat_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/overweight-and-obesity-management-pdf-66143959958725_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/constipation-in-children-and-young-people-diagnosis-and-management-pdf-975757753285_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/obesity-in-adults-prevention-and-lifestyle-weight-management-programmes-pdf-75545293071301_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/fatty acid intake.pdf",
    "C:/Users/Ayush Jindal/Downloads/WHO guideline for physics activity.pdf",
    "C:/Users/Ayush Jindal/Downloads/physical-activity-exercise-referral-schemes-pdf-1996418406085_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/bipolar-disorder-assessment-and-management-pdf-35109814379461_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/obesity-clinical-assessment-and-management-pdf-75545363615173_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/IND301-20250415_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/hypertension.pdf",
    "C:/Users/Ayush Jindal/Downloads/NICE 1.pdf",
    "C:/Users/Ayush Jindal/Downloads/NICD guideline mental health.pdf",
    "C:/Users/Ayush Jindal/Downloads/adolescent-health.pdf",
    "C:/Users/Ayush Jindal/Downloads/B1.pdf",
]

# --- Text Extraction Function ---
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file page by page.
    Includes error handling for missing files or extraction issues.
    """
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            print(f"  Extracting text from {num_pages} pages...")
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                try:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text
                except Exception as page_e:
                    print(f"    Warning: Could not extract text from page {page_num + 1} of {os.path.basename(pdf_path)}: {page_e}")
                    text += "[PAGE_TEXT_EXTRACTION_ERROR]"
        print(f"Successfully extracted raw text from: {os.path.basename(pdf_path)}")
    except FileNotFoundError:
        print(f"Error: PDF file not found at '{pdf_path}'. Please check the path and filename.")
    except Exception as e:
        print(f"Error opening or reading PDF '{pdf_path}': {e}")
    return text

# --- Initial Text Cleaning Function ---
def clean_initial_pdf_text(text):
    """
    Performs initial cleaning on raw extracted PDF text.
    Removes common PDF artifacts and standardizes formatting.
    """
    # 1. Remove source tags like or 

    # 2. Remove common PDF rendering artifacts like (cid:xx)
    text = re.sub(r'\(cid:\d+\)', '', text)

    # 3. Remove specific repeating headers/footers based on observation of your PDFs
    text = re.sub(r'Routine Blood\n Results\n Explained\n', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Routine Blood Results Explained', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Blood Results in\n Clinical Practice', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Blood Results in Clinical Practice', '', text, flags=re.IGNORECASE)
    text = re.sub(r'A practical guide\n to interpreting blood\n test results', '', text, flags=re.IGNORECASE)

    # 4. Remove page numbers. This is heuristic and targets various page number patterns.
    text = re.sub(r'^\s*\d+\s*(\n|$)', '\n', text, flags=re.MULTILINE)
    text = re.sub(r'(\n|^)\s*\d+\s*(\n|$)', '\n', text)
    text = re.sub(r'Page \d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Chapter \d+\s*\d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'^\s*\d{1,3}\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'^\s*(viii|ix|x|xi|xii|xiii|xiv|xv|xvi|xvii|xviii|xix|xx)\s*$', '', text, flags=re.IGNORECASE | re.MULTILINE)

    # 5. Remove TOC-like entries that remain, or prefaces/abbreviations
    text = re.sub(r'^\s*(Contents|Preface|Abbreviations|Introduction|Part \d+|Chapter \d+).*?\b(?:vii|viii|ix|x|xi|xii|\d+)\s*$', '', text, flags=re.IGNORECASE | re.MULTILINE)

    # 6. Standardize whitespace: Replace multiple whitespaces (including newlines) with single space
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'([.?!])\s*(?=[A-Z0-9]|$)', r'\1\n\n', text) # Add double newline after sentences
    text = re.sub(r'\n{3,}', '\n\n', text) # Reduce excessive newlines

    # 7. Remove hyphenation from words split across lines
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)

    # 8. Clean specific table artifacts from contents that might be extracted oddly
    text = re.sub(r'"([^"]+)"\s*,\s*"([^"]+)"', r'\1 \2', text)
    text = re.sub(r'Objectives and Scope\n', '', text)
    text = re.sub(r'Table \d+\.\d+:.*?\n', '', text, flags=re.IGNORECASE)
    text = re.sub(r'The following table:', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Mode LastWriteTime Length ---- ------------- ------', '', text)

    return text

# --- Main Processing Loop ---
cleaned_texts_by_book = {}

for pdf_file_name in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf_file_name)
    print(f"\n--- Processing: {pdf_file_name} ---")

    raw_text = extract_text_from_pdf(pdf_path)
    if raw_text:
        cleaned_text = clean_initial_pdf_text(raw_text)
        cleaned_texts_by_book[pdf_file_name] = cleaned_text
        print(f"Initial cleaning complete for {pdf_file_name}. Cleaned length: {len(cleaned_text)} characters.")
    else:
        print(f"Skipping {pdf_file_name} due to extraction error or empty content.")

print("\n--- Initial Text Cleaning Summary ---")
for book_name, text_content in cleaned_texts_by_book.items():
    print(f"Book: {book_name[:35]}... | Length: {len(text_content)} chars")
    print("Sample (first 1000 chars):\n", text_content[:1000])
    print("-" * 50)


--- Processing: C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf ---
  Extracting text from 172 pages...
Successfully extracted raw text from: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf
Initial cleaning complete for C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf. Cleaned length: 264579 characters.

--- Processing: C:/Users/Ayush Jindal/Downloads/Wallach.pdf ---
  Extracting text from 1884 pages...
Successfully extracted raw text from: Wallach.pdf
Initial cleaning complete for C:/Users/Ayush Jindal/Downloads/Wallach.pdf. Cleaned length: 3245859 characters.

--- Processing: C:/Users/Ayush Jindal/Downloads/Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf ---
  Extracting text from 119 pages...
Successfully extracted raw text f

In [18]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\Ayush
[nltk_data]     Jindal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to C:\Users\Ayush
[nltk_data]     Jindal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to C:\Users\Ayush
[nltk_data]     Jindal\AppData\Roaming\nltk_data...


True

In [3]:
import re
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from langchain.text_splitter import RecursiveCharacterTextSplitter

# cleaned_texts_by_book is already populated from your successful Step 1 run.
# The dummy data fallback is not strictly needed if Step 1 was successful,
# but it's kept here for a complete, runnable block in isolation.
if 'cleaned_texts_by_book' not in locals() or not cleaned_texts_by_book:
    print("Error: 'cleaned_texts_by_book' dictionary not found or empty. Please ensure Step 1 was executed successfully.")
    # Dummy data fallback for demonstration if the dictionary isn't populated
    # These strings are simplified to allow the code to run if actual PDFs aren't extracted
    cleaned_texts_by_book = {
        'ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf': """INTRODUCTION: ROUTINE BLOOD RESULTS EXPLAINED. This book covers haematology, immunology, and biochemistry. Blood tests provide crucial information for medical decisions. Each section has dedicated examples. Knowledge needs practice, so case studies are included. Labs may differ in where tests are performed. Units like g/L or mmol/L are common. Reference ranges are important but vary by hospital and population. Near patient testing offers rapid results. This is Part 1: Haematology and Blood Transfusion. Objectives and Scope are listed in Table 1. The lab provides info on blood cells and clotting. Blood Transfusion gives components like red cells. All samples must be labeled correctly. Basic blood tubes are used: EDTA for FBC, sodium citrate for coagulation. ESR may need its own tube. Immunologists use serum or plasma. Failure to use the correct tube will result in a repeat request. Haematology has red cell, white cell, and coagulation areas. Anaemia, infection, neoplasia, thrombosis, and hemorrhage are major problems. Chapter 1: The Red Blood Cell. Key words: Haemoglobin (Hb), Red blood cell count (RBCC), Haematocrit (Hct), Mean cell volume (MCV), Mean cell haemoglobin (MCH), Mean cell haemoglobin concentration (MCHC), Red cell distribution width (RDW), Erythrocyte sedimentation rate (ESR). Key pathological expressions: Anaemia, Polycythaemia, Thalassaemia, Sickle cell disease. Haemoglobin is a protein designed to carry oxygen. Reference range varies. Red blood cells lack a nucleus. Hct expresses proportion of whole blood by cells. MCV is the size of average red blood cell. MCH is average amount of Hb in cell. MCHC merges MCV and MCH. These are red cell indices. RDW quantifies variation in cell size. ESR is a global score of physical aspects of blood. Increased ESR has many causes. Other haematology tests include plasma viscosity, iron, ferritin, and B12. Reticulocytes are immature red cells. What are red cells for? To carry oxygen. Insufficient oxygen (hypoxia) causes symptoms like tiredness. Hypoxia can be due to lung disease or heart failure. Anaemia means reduced oxygen carrying capacity. It can be defined by low Hb and symptoms. More serious signs include jaundice. Anaemia classification: depressed red cell production (cancer, aplastic anaemia, drugs), diet deficiency (iron, B12, folate), loss of mature red cells (drugs, fevers, auto-immunity, bleeding), haemoglobinopathy (sickle cell, thalassaemia), disease in other organs (liver). Case report: microcytic anaemia.
    """,
        'Wallach.pdf': """Wallach’s Interpretation of Diagnostic Tests. This book provides a comprehensive guide to diagnostic tests. It covers a wide array of laboratory medicine disciplines, including clinical chemistry, hematology, immunology, and microbiology. The text details the principles behind various tests, their clinical applications, and interpretation of results in the context of different diseases. It emphasizes the importance of understanding normal ranges and how deviations indicate pathology. Key sections include discussions on organ system diseases, such as kidney, liver, and cardiovascular disorders, and how laboratory markers aid in their diagnosis and management. The book also covers infectious diseases, endocrine disorders, and cancer markers. It highlights the role of laboratory data in patient care, from initial diagnosis to monitoring treatment efficacy. Special considerations for different patient populations, including pediatrics and geriatrics, are also addressed. The significance of various blood tests, such as those for electrolytes, glucose, lipids, and blood cell counts, is explained in detail. Interpretation of complex lab panels and the impact of medications on test results are also central themes. It is a vital resource for understanding the diagnostic utility of laboratory tests in clinical practice.
        """,
        'Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf': """Blood Results in Clinical Practice. This book is a practical guide to interpreting blood test results. It aims to demystify complex topics for healthcare professionals, students, and patients. The book introduces a 10-point system and uses storytelling to explain abstract concepts, like 'fire engines' for neutrophils. It emphasizes considering four key questions when interpreting results: how far out of range is it, does it make sense, do family groups support the result, and is it an important test? It details the difference between biochemistry (chemicals) and haematology (cells). The book covers blood collection methods, haemolysed samples, and reference ranges, noting that ranges are specific to machines and local settings. It discusses clinical sensitivity and specificity, distinguishing true/false positives/negatives. Clinical decision limits (CDLs) are introduced for managing patient care based on specific result thresholds. Chapters delve into the full blood count (FBC), coagulation, inflammatory markers, the immune system, chronic disease markers (diabetes, cholesterol), thyroid function, bone profile, and renal function (urea and electrolytes). It explains the clinical implications of various markers like CRP, ESR, PV, ALT, AST, bilirubin, GGT, albumin, and coagulation factors (INR, D-dimer). The book highlights the interconnectedness of organ systems (e.g., kidney and red cell production, liver and inflammation). It also addresses complex scenarios like anaemia of chronic disease and neuropathies. Transfusion testing, ABO/Rh blood groups, and transfusion reactions are covered. The afterword emphasizes the importance of systematic planning in interpreting blood tests.
        """,
        'guidelines-on-mental-health-promotive-and-preventive-interventions-for-adolescents-hat_copy.pdf': """Guidelines on mental health promotive and preventive interventions for adolescents. This WHO guideline focuses on helping adolescents thrive. It provides evidence-based recommendations for mental health promotion and prevention of mental disorders in young people. Key areas include interventions across different settings, such as schools, communities, and digital platforms. It addresses topics like resilience building, social emotional learning, early identification of mental health risks, and support systems for adolescents. The guideline emphasizes a multi-sectoral approach involving health, education, and social services. It also covers the importance of family and peer support in adolescent mental health.
        """,
        'overweight-and-obesity-management-pdf-66143959958725_copy.pdf': """Overweight and obesity management. This NICE guideline provides recommendations for the identification, assessment, and management of overweight and obesity. It covers both adults and children, offering guidance on lifestyle weight management programs. The guideline addresses various interventions, including dietary advice, physical activity recommendations, and behavioral change strategies. It emphasizes personalized care and consideration of individual needs. It also discusses the role of healthcare professionals in supporting people to achieve a healthy weight and reduce obesity-related health risks.
        """,
        'constipation-in-children-and-young-people-diagnosis-and-management-pdf-975757753285_copy.pdf': """Constipation in children and young people: diagnosis and management. This NICE clinical guideline provides recommendations for the diagnosis and management of constipation in children and young people. It covers topics such as assessment of symptoms, identifying underlying causes, and various treatment options including dietary changes, lifestyle advice, and pharmacological interventions. The guideline emphasizes a child-centered approach and effective communication with families. It also addresses follow-up care and management of chronic constipation.
        """,
        'obesity-in-adults-prevention-and-lifestyle-weight-management-programmes-pdf-75545293071301_copy.pdf': """Obesity in adults: prevention and lifestyle weight management programmes. This NICE quality standard focuses on preventing obesity and providing effective lifestyle weight management programs for adults. It outlines key quality statements related to promoting healthy eating, increasing physical activity, and delivering structured weight management interventions. The standard emphasizes a public health approach to address obesity at a population level, as well as individual support for those who are overweight or obese.
        """,
        'fatty acid intake.pdf': """Saturated fatty acid and trans-fatty acid intake for adults and children WHO guideline. This WHO guideline provides recommendations on the intake of saturated fatty acids and trans-fatty acids for adults and children. It outlines dietary advice to reduce the risk of noncommunicable diseases. The guideline emphasizes limiting the consumption of unhealthy fats and promoting healthier dietary patterns globally. It also discusses the impact of these fatty acids on cardiovascular health and overall well-being.
        """,
        'WHO guideline for physics activity.pdf': """WHO guidelines on physical activity and sedentary behaviour. This WHO guideline provides recommendations on physical activity and sedentary behavior for all age groups, including children, adolescents, adults, and older adults. It outlines recommended levels of physical activity to improve health outcomes and reduce the risk of noncommunicable diseases. The guideline emphasizes the benefits of regular physical activity for physical and mental health, and provides guidance on reducing sedentary time.
        """,
        'physical-activity-exercise-referral-schemes-pdf-1996418406085_copy.pdf': """Physical activity: exercise referral schemes. This NICE public health guideline provides recommendations on exercise referral schemes. It evaluates the effectiveness of these schemes in promoting physical activity among individuals who are inactive or have certain health conditions. The guideline addresses the design, delivery, and evaluation of exercise referral programs, aiming to improve public health outcomes related to physical activity.
        """,
        'bipolar-disorder-assessment-and-management-pdf-35109814379461_copy.pdf': """Bipolar disorder: assessment and management. This NICE clinical guideline provides recommendations for the assessment and management of bipolar disorder. It covers diagnosis, pharmacological treatments, psychological interventions, and long-term management strategies. The guideline emphasizes personalized care and collaborative decision-making with individuals and their families. It also addresses management during different phases of the illness and specific considerations for different age groups.
        """,
        'obesity-clinical-assessment-and-management-pdf-75545363615173_copy.pdf': """Obesity: clinical assessment and management. This NICE quality standard outlines key quality statements for the clinical assessment and management of obesity. It covers topics such as initial assessment, personalized care planning, interventions including dietary and physical activity advice, pharmacological treatments, and bariatric surgery. The standard aims to improve the quality of care for individuals living with obesity and reduce associated health complications.
        """,
        'IND301-20250415_copy.pdf': """Diabetes: annual BMI (children T2DM). This NICE indicator specifies the proportion of children and young people under 18 with type 2 diabetes who have had their BMI recorded in the previous 12 months. It is a network/system level indicator aiming to monitor the performance of providers in managing childhood diabetes. The rationale emphasizes that monitoring BMI helps identify weight gain and prompts appropriate action.
        """,
        'hypertension.pdf': """Hypertension in adults: diagnosis and management. This NICE guideline provides recommendations for the diagnosis and management of hypertension (high blood pressure) in adults. It covers blood pressure measurement, assessment of cardiovascular risk, lifestyle interventions, and pharmacological treatments. The guideline aims to reduce the risk of cardiovascular events and improve long-term health outcomes for individuals with hypertension.
        """,
        'NICE 1.pdf': """Obesity in children and young people: prevention and lifestyle weight management programmes. This NICE quality standard outlines key quality statements for preventing obesity and delivering lifestyle weight management programs for children and young people. It covers topics such as early intervention, healthy eating, physical activity, and family-based approaches to promote healthy weight development.
        """,
        'NICD guideline mental health.pdf': """Antisocial behaviour and conduct disorders in children and young people: recognition and management. This NICE clinical guideline provides recommendations for recognizing and managing antisocial behavior and conduct disorders in children and young people. It covers assessment, interventions, and support for individuals and their families. The guideline emphasizes early identification and tailored approaches to improve outcomes.
        """,
        'adolescent-health.pdf': """Adolescent Health: A Multidisciplinary Approach to Theory, Research, and Intervention. This academic book provides a comprehensive multidisciplinary approach to adolescent health. It covers theories, research, and interventions related to the physical, mental, and social well-being of adolescents. Topics include developmental stages, common health risks, health behaviors, mental health issues, and strategies for effective interventions.
        """,
        'B1.pdf': """Harrison's Principles Of Internal Medicine 15th Edition. This is a comprehensive medical textbook covering a vast range of diseases, their diagnosis, pathophysiology, and management. It is divided into numerous parts covering various organ systems, infectious diseases, oncology, hematology, endocrinology, neurology, and environmental hazards. It provides detailed explanations of clinical medicine, manifestations of disease, genetics, pharmacology, and nutrition.
        """
    }


# --- Advanced Text Preprocessing and Chunking Function ---
def clean_and_chunk_text(book_text, book_title, chunk_size=500, chunk_overlap=100):
    """
    Applies further cleaning and splits text into smaller, overlapping chunks suitable for LLMs.
    """
    # Apply light preprocessing
    # Remove chapter/page number patterns from headers/footers that might remain
    processed_text = re.sub(r'Chapter \d+.*?(\d+|vii|ix|x|xi)\s*$', '', book_text, flags=re.MULTILINE)
    processed_text = re.sub(r'(\n|^)\s*(Preface|Abbreviations|Introduction|Part \d+|Contents)\s*$', '', processed_text, flags=re.MULTILINE | re.IGNORECASE)
    processed_text = re.sub(r'\s+', ' ', processed_text).strip() # Standardize whitespace
    processed_text = re.sub(r'([.?!])\s*(?=[A-Z0-9]|$)', r'\1\n\n', processed_text) # Add double newline after sentences
    processed_text = re.sub(r'\n{3,}', '\n\n', processed_text) # Re-reduce excessive newlines after sentence splitting

    # Initialize text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""] # Prioritize splitting by larger units first
    )

    # Split the document into chunks
    chunks = text_splitter.split_text(processed_text)
    
    # Add metadata to each chunk
    chunks_with_metadata = []
    for i, chunk in enumerate(chunks):
        chunks_with_metadata.append({
            'book_name': book_title,
            'chunk_id': f"{os.path.splitext(book_title)[0].replace(' ', '_').replace('.', '').replace('__', '_')}_chunk_{i:04d}", # Cleaned ID for potential vector DB use
            'chunk_text': chunk,
            'chunk_length': len(chunk)
            # 'start_char': book_text.find(chunk) # Approximate start char - may not be accurate after extensive cleaning
        })
    return chunks_with_metadata

# --- Main Chunking Process ---
all_processed_chunks = []

for book_name, text_content in cleaned_texts_by_book.items():
    print(f"\n--- Chunking text for: {book_name} ---")
    chunks_for_book = clean_and_chunk_text(text_content, book_name, chunk_size=500, chunk_overlap=100) # Adjust chunk_size/overlap as needed
    all_processed_chunks.extend(chunks_for_book)
    print(f"Generated {len(chunks_for_book)} chunks for {book_name}.")

# Create a DataFrame of all chunks with their metadata
df_book_chunks = pd.DataFrame(all_processed_chunks)

print("\n--- Chunks DataFrame Head (from all books) ---")
print(df_book_chunks.head())
print(f"\nTotal number of chunks generated: {len(df_book_chunks)}")
print(f"Average chunk length: {df_book_chunks['chunk_length'].mean():.2f} characters.")

# Example: Display a few sample chunks for review
if not df_book_chunks.empty:
    print("\n--- Sample Chunks for Review ---")
    for i in range(min(3, len(df_book_chunks))): # Display first 3 chunks
        print(f"\nChunk ID: {df_book_chunks['chunk_id'].iloc[i]}")
        print(f"Book: {df_book_chunks['book_name'].iloc[i]}")
        print(f"Text ({df_book_chunks['chunk_length'].iloc[i]} chars):")
        print(df_book_chunks['chunk_text'].iloc[i])
        print("-" * 70)


--- Chunking text for: C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf ---
Generated 684 chunks for C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf.

--- Chunking text for: C:/Users/Ayush Jindal/Downloads/Wallach.pdf ---
Generated 8485 chunks for C:/Users/Ayush Jindal/Downloads/Wallach.pdf.

--- Chunking text for: C:/Users/Ayush Jindal/Downloads/Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf ---
Generated 485 chunks for C:/Users/Ayush Jindal/Downloads/Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf.

--- Chunking text for: C:/Users/Ayush Jindal/Downloads/guidelines-on-mental-health-promotive-and-preventive-interventions-for-adolescents-hat_copy.pdf ---
Generated 902 chunks for C:/U

In [21]:
!pip install torch sentence-transformers


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch # Ensure torch is imported, as it's a dependency of sentence_transformers

# df_book_chunks is already available from your successfully completed Step 2.
# The dummy data fallback is kept here for a complete, runnable block in isolation,
# but your actual df_book_chunks will be used.
if 'df_book_chunks' not in locals() or df_book_chunks.empty:
    print("Error: 'df_book_chunks' DataFrame is empty or not found. Please ensure Step 2 was executed successfully.")
    # Fallback to dummy data for demonstration if df_book_chunks is empty
    df_book_chunks = pd.DataFrame({
        'book_name': ['Dummy Book 1.pdf', 'Dummy Book 2.pdf'],
        'chunk_id': ['dummy_book_1_chunk_0001', 'dummy_book_2_chunk_0001'],
        'chunk_text': [
            'This is a sample text chunk about medical results and interpretations for student health reports.',
            'Another chunk discussing the importance of physical activity guidelines and mental health interventions for adolescents.'
        ],
        'chunk_length': [100, 150]
    })
    print("Dummy df_book_chunks created for demonstration.")


# Initialize a Sentence Transformer model
print("Loading Sentence Transformer model...")
# 'all-MiniLM-L6-v2' is a good balance of size and performance for general-purpose embeddings.
# This will load the model into memory.
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded.")

# Generate embeddings for each chunk
print(f"\nGenerating embeddings for {len(df_book_chunks)} chunks. This will take a significant amount of time given the large number of chunks...")
# Convert the 'chunk_text' column to a list for the model's input
chunk_texts_list = df_book_chunks['chunk_text'].tolist()

# Generate embeddings. The model.encode method handles batching and GPU usage automatically if available.
# The show_progress_bar=True argument will display a progress bar.
# convert_to_tensor=False ensures the output is NumPy arrays, which is convenient for pandas.
chunk_embeddings = model.encode(chunk_texts_list, show_progress_bar=True, convert_to_tensor=False)

print(f"Generated embeddings with shape: {chunk_embeddings.shape}")

# Add the embeddings as a new column to the DataFrame
# Storing NumPy arrays directly in a DataFrame column is efficient.
df_book_chunks['embedding'] = list(chunk_embeddings)

print("\n--- Final Chunks DataFrame Head with Embeddings ---")
print(df_book_chunks.head())
print(f"\nFinal DataFrame shape with embeddings: {df_book_chunks.shape}")

# You can now optionally save this vectorized data to a file.
# Parquet is highly recommended for its efficiency with large datasets and numerical arrays.
# Make sure you have 'pyarrow' installed for this: !pip install pyarrow
try:
    df_book_chunks.to_parquet('vectorized_medical_book_chunks.parquet', index=False)
    print("\nVectorized data successfully saved as 'vectorized_medical_book_chunks.parquet'")
    # Get current working directory to show full path where file is saved
    import os
    print(f"File saved to: {os.path.join(os.getcwd(), 'vectorized_medical_book_chunks.parquet')}")
except Exception as e:
    print(f"Error saving to Parquet: {e}")
    print("Please ensure 'pyarrow' or 'fastparquet' is installed for Parquet support. You may have to install it: !pip install pyarrow")
    print("Attempting to save as CSV as a fallback (embeddings will be stored as strings, which is less ideal).")
    try:
        df_book_chunks.to_csv('vectorized_medical_book_chunks.csv', index=False)
        print("Vectorized data saved as 'vectorized_medical_book_chunks.csv'")
    except Exception as csv_e:
        print(f"Error saving to CSV: {csv_e}")

# This `df_book_chunks` DataFrame, with its 'book_name', 'chunk_id', 'chunk_text', and 'embedding' columns,
# is the clean, vectorized data suitable for feeding into an an LLM system for tasks like:
# - Storing in a Vector Database for Retrieval Augmented Generation (RAG).
# - Directly using for similarity search to find relevant information based on user queries.
# - Potentially fine-tuning an LLM (though this requires much more data and computational resources).

C:\Users\Ayush Jindal\Desktop\ChatBotProject\chatbot_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Sentence Transformer model...
Model loaded.

Generating embeddings for 58762 chunks. This will take a significant amount of time given the large number of chunks...


Batches: 100%|█| 1837/1837 [26:58<00:00,  1


Generated embeddings with shape: (58762, 384)

--- Final Chunks DataFrame Head with Embeddings ---
                                           book_name  \
0  C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD ...   
1  C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD ...   
2  C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD ...   
3  C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD ...   
4  C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD ...   

                                            chunk_id  \
0  C:/Users/Ayush_Jindal/Downloads/ROUTINE_BLOOD_...   
1  C:/Users/Ayush_Jindal/Downloads/ROUTINE_BLOOD_...   
2  C:/Users/Ayush_Jindal/Downloads/ROUTINE_BLOOD_...   
3  C:/Users/Ayush_Jindal/Downloads/ROUTINE_BLOOD_...   
4  C:/Users/Ayush_Jindal/Downloads/ROUTINE_BLOOD_...   

                                          chunk_text  chunk_length  \
0  Routine Blood Results Explained Routine Blood ...           495   
1  book is available from the British Library Cop...           128   
2  No pa rt of th

In [24]:
!pip install pyarrow

   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   - -------------------------------------- 0.8/25.7 MB 7.6 MB/s eta 0:00:04
   -- ------------------------------------- 1.8/25.7 MB 4.5 MB/s eta 0:00:06
   ---- ----------------------------------- 2.6/25.7 MB 4.2 MB/s eta 0:00:06
   ----- ---------------------------------- 3.4/25.7 MB 4.1 MB/s eta 0:00:06
   ------ --------------------------------- 4.2/25.7 MB 4.0 MB/s eta 0:00:06
   ------- -------------------------------- 5.0/25.7 MB 4.0 MB/s eta 0:00:06
   -------- ------------------------------- 5.8/25.7 MB 4.0 MB/s eta 0:00:06
   ---------- ----------------------------- 6.6/25.7 MB 3.9 MB/s eta 0:00:05
   ----------- ---------------------------- 7.3/25.7 MB 3.9 MB/s eta 0:00:05
   ----------- ---------------------------- 7.6/25.7 MB 3.9 MB/s eta 0:00:05
   ------------- -------------------------- 8.7/25.7 MB 3.8 MB/s eta 0:00:05
   -------------- ------------------------- 9.2/25.7 MB 3.7 MB/s eta 0:00:05
   ---


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
